# Price Action Strategy - by ChatGPT - Backtesting

## Import Library

In [1]:
import warnings
warnings.filterwarnings('ignore')
#
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy
# from backtesting._plotting import set_bokeh_output
# set_bokeh_output(notebook=False)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120

Loading BokehJS ...

## Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
csv_file = str(current_dir.parent) + '/VN30F1M/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print(csv_file)
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
data = dataset.copy()

In [4]:
data = data[data.index > '2024-11-01 00:00:00']

In [5]:
# Hàm tìm Demand Zone (nến xanh có khối lượng cao nhất trong 100 nến)
def find_demand_zone(df, window=100):
    demand_zones = []
    for i in range(window, len(df)):
        sub_df = df[i-window:i]
        max_idx = sub_df[sub_df['Close'] > sub_df['Open']]['Volume'].idxmax()
        if pd.notna(max_idx):
            demand_zones.append(df.loc[max_idx, ['Low', 'High']])
    return pd.DataFrame(demand_zones)

# Hàm tìm Supply Zone (nến đỏ có khối lượng cao nhất trong 100 nến)
def find_supply_zone(df, window=100):
    supply_zones = []
    for i in range(window, len(df)):
        sub_df = df[i-window:i]
        max_idx = sub_df[sub_df['Close'] < sub_df['Open']]['Volume'].idxmax()
        if pd.notna(max_idx):
            supply_zones.append(df.loc[max_idx, ['Low', 'High']])
    return pd.DataFrame(supply_zones)

# Hàm nhận diện mô hình nến đảo chiều
def detect_reversal_patterns(df):
    signals = []
    for i in range(2, len(df)):
        if (df['Close'][i] > df['Open'][i] and df['Close'][i-1] < df['Open'][i-1] and
            df['Close'][i] > df['Open'][i-1] and df['Open'][i] < df['Close'][i-1]):
            signals.append(('Bullish Engulfing', i))
        elif (df['Close'][i] < df['Open'][i] and df['Close'][i-1] > df['Open'][i-1] and
              df['Close'][i] < df['Open'][i-1] and df['Open'][i] > df['Close'][i-1]):
            signals.append(('Bearish Engulfing', i))
        elif (df['Close'][i-1] < df['Open'][i-1] and df['Close'][i] > df['Open'][i] and
              df['Close'][i] >= (df['Open'][i-1] + df['Close'][i-1]) / 2):
            signals.append(('Piercing Line', i))
        elif (df['Close'][i-1] > df['Open'][i-1] and df['Close'][i] < df['Open'][i] and
              df['Close'][i] <= (df['Open'][i-1] + df['Close'][i-1]) / 2):
            signals.append(('Dark Cloud Cover', i))
        elif (df['Close'][i-2] > df['Open'][i-2] and df['Close'][i-1] < df['Open'][i-1] and
              df['Close'][i] < df['Open'][i] and df['Close'][i] < df['Close'][i-2]):
            signals.append(('Evening Star', i))
        elif (df['Close'][i-2] < df['Open'][i-2] and df['Close'][i-1] > df['Open'][i-1] and
              df['Close'][i] > df['Open'][i] and df['Close'][i] > df['Close'][i-2]):
            signals.append(('Morning Star', i))
    return signals

# Hàm xác định tín hiệu giao dịch
def find_signals(df, demand_zones, supply_zones, reversal_patterns):
    signals = []
    for pattern, i in reversal_patterns:
        price = df['Close'][i]
        if any(df['Low'][i] >= dz['Low'] and df['High'][i] <= dz['High'] for _, dz in demand_zones.iterrows()):
            if pattern in ['Bullish Engulfing', 'Piercing Line', 'Morning Star']:
                signals.append(('Long', i))
        if any(df['Low'][i] >= sz['Low'] and df['High'][i] <= sz['High'] for _, sz in supply_zones.iterrows()):
            if pattern in ['Bearish Engulfing', 'Dark Cloud Cover', 'Evening Star']:
                signals.append(('Short', i))
    return signals

In [6]:
demand_zones = find_demand_zone(data)

In [7]:
demand_zones

,Low,High
2024-11-04 13:45:00,1319.4,1323.3
2024-11-04 13:45:00,1319.4,1323.3
2024-11-04 13:45:00,1319.4,1323.3
2024-11-04 13:45:00,1319.4,1323.3
2024-11-04 13:45:00,1319.4,1323.3
...,...,...
2025-02-14 09:00:00,1345.2,1348.9
2025-02-14 09:00:00,1345.2,1348.9
2025-02-14 09:00:00,1345.2,1348.9
2025-02-14 09:00:00,1345.2,1348.9


In [8]:
supply_zones = find_supply_zone(data)

In [9]:
supply_zones

,Low,High
2024-11-01 14:15:00,1326.6,1331.1
2024-11-01 14:15:00,1326.6,1331.1
2024-11-01 14:15:00,1326.6,1331.1
2024-11-01 14:15:00,1326.6,1331.1
2024-11-01 14:15:00,1326.6,1331.1
...,...,...
2025-02-12 14:25:00,1332.8,1335.8
2025-02-14 13:25:00,1342.1,1345.5
2025-02-14 13:25:00,1342.1,1345.5
2025-02-14 13:25:00,1342.1,1345.5


In [10]:
reversal_patterns = detect_reversal_patterns(data)

In [12]:
test_df = find_signals(data, demand_zones, supply_zones, reversal_patterns)

KeyboardInterrupt: 

In [ ]:
test_df

In [13]:
# Chiến lược giao dịch dựa trên Price Action
class PriceActionStrategy(Strategy):
    def init(self):
        pass
    
    def next(self):
        df = self.data.df
        demand_zones = find_demand_zone(df)
        supply_zones = find_supply_zone(df)
        reversal_patterns = detect_reversal_patterns(df)
        signals = find_signals(df, demand_zones, supply_zones, reversal_patterns)
        
        for signal, i in signals:
            price = self.data.Close[i]
            if signal == 'Long':
                sl = min(self.data.Low[i-1], self.data.Low[i])
                tp = price + 3 * (price - sl)
                self.buy(sl=sl, tp=tp)
            elif signal == 'Short':
                sl = max(self.data.High[i-1], self.data.High[i])
                tp = price - 3 * (sl - price)
                self.sell(sl=sl, tp=tp)

In [14]:
# Chạy Backtest
bt = Backtest(data, PriceActionStrategy, cash=10000, commission=.002)
stats = bt.run()
bt.plot()

AttributeError: 'Rolling' object has no attribute 'idxmax'